In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StringType, StructType, StructField
from pyspark.conf import SparkConf

In [2]:
kafka_bootstrap_servers = 'peter-kafka01.foo.bar:9092,peter-kafka02.foo.bar:9092,peter-kafka03.foo.bar:9092'
topic = 'airportcodes'

conf = (
SparkConf()
    .setAppName("MY_kafka") # replace with your desired name
    .set("spark.jars.packages", \
         "org.apache.commons:commons-pool2:2.11.1"\
         ",org.apache.kafka:kafka-clients:3.4.0"\
         ",org.apache.spark:spark-protobuf_2.12:3.4.1"\
         ",org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1"\
         ",org.apache.spark:spark-token-provider-kafka-0-10_2.12:3.4.1"\
        )
    # .set("spark.sql.shuffle.partitions", "4") # default is 200 partitions which is too many for local
    # .setMaster("local[*]") # replace the * with your desired number of cores. * for use all.
    .setMaster("spark://34.125.136.103:30077")
    .set("spark.driver.host", "20.109.131.18") # 원격 실행 시 exited with code 1 exitStatus 1(무한재시작) 현상 방지용
    .set("spark.driver.hostname", "20.109.131.18") # 원격 실행 시 exited with code 1 exitStatus 1(무한재시작) 현상 방지용
    # .set("spark.driver.port", "18000") # 원격 실행 시 exited with code 1 exitStatus 1(무한재시작) 현상 방지용
    .set("spark.driver.bindAddress", "0.0.0.0") # 원격 실행 시 exited with code 1 exitStatus 1(무한재시작) 현상 방지용
    # .set("spark.ui.port", "18001") # 원격 실행 시 exited with code 1 exitStatus 1(무한재시작) 현상 방지용
    # .set("spark.blockManager.port", "18002") # 원격 실행 시 exited with code 1 exitStatus 1(무한재시작) 현상 방지용
)

In [3]:
spark = SparkSession.builder \
    .config(conf=conf)\
    .getOrCreate()

:: loading settings :: url = jar:file:/root/miniconda3/envs/spark341/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.commons#commons-pool2 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.spark#spark-protobuf_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-token-provider-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b033d62d-cabf-482c-8f2a-35bde00a6aff;1.0
	confs: [default]
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.4.0 in central
	found com.github.luben#zstd-jni;1.5.2-1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found org.apache.spark#spark-protobuf_2.12;3.4.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.spark#spark-sql-kafka-0-10_2

In [4]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", topic) \
    .option("startingOffsets", "earliest") \
    .option("maxOffsetsPerTrigger", 100) \
    .load()

# 메시지의 value를 JSON으로 파싱
schema = StructType([
    StructField("ident", StringType(), True),
    StructField("type", StringType(), True),
    StructField("name", StringType(), True),
    StructField("elevation_ft", StringType(), True),
    StructField("continent", StringType(), True),
    StructField("iso_country", StringType(), True),
    StructField("iso_region", StringType(), True),
    StructField("municipality", StringType(), True),
    StructField("gps_code", StringType(), True),
    StructField("iata_code", StringType(), True),
    StructField("local_code", StringType(), True),
    StructField("coordinates", StringType(), True),
])

json_df = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

# 스트림 출력 설정
query = json_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

24/04/30 22:40:05 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-becb8f89-7b42-4a4b-bf9b-6ca7bd3ae486. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/30 22:40:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [5]:
query.awaitTermination()

24/04/30 22:40:06 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
24/04/30 22:40:26 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/root/miniconda3/envs/spark341/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/root/miniconda3/envs/spark341/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/root/miniconda3/envs/spark341/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
spark.stop()